# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\Female\GeneCpG\Cognition
df = pd.read_csv('CognitionCateg_Methylome_Female.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\Female\GeneCpG\Cognition
(150, 123024)


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,Montreal_Cognitive,cg08012045,cg08300930,cg00134295,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,214,200991620021_R03C01,3002,2,2,1,0,0.952757,0.970714,0.227504,...,0.918107,0.923478,0.902558,0.775472,0.832588,0.259813,0.306235,0.836966,0.063255,0.888196
1,215,200989060105_R03C01,3003,2,2,1,1,0.948345,0.966610,0.209896,...,0.844439,0.906448,0.901592,0.760667,0.923298,0.247964,0.251046,0.865364,0.092802,0.830034
2,216,200989060162_R04C01,3006,2,2,1,0,0.961376,0.974806,0.197703,...,0.898837,0.915001,0.887093,0.817672,0.879020,0.186693,0.189148,0.748168,0.080457,0.869580


In [3]:
df1 = df.drop(['Unnamed: 0','Sentrix','PATNO','HYS','Gender','Montreal_Cognitive'], axis=1)
df1.head(2)

,APPRDX,cg08012045,cg08300930,cg00134295,cg10537824,cg04990650,cg03379001,cg02215627,cg06699201,cg18100935,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,1,0.952757,0.970714,0.227504,0.860591,0.836116,0.863656,0.885796,0.741853,0.879686,...,0.918107,0.923478,0.902558,0.775472,0.832588,0.259813,0.306235,0.836966,0.063255,0.888196
1,1,0.948345,0.966610,0.209896,0.826954,0.798853,0.877657,0.846453,0.778225,0.902908,...,0.844439,0.906448,0.901592,0.760667,0.923298,0.247964,0.251046,0.865364,0.092802,0.830034


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(30))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5333333333333334
2
Mean accuracy: 0.65
3
Mean accuracy: 0.4833333333333332
4
Mean accuracy: 0.41666666666666663
5
Mean accuracy: 0.5
6
Mean accuracy: 0.4333333333333333
7
Mean accuracy: 0.4499999999999999
8
Mean accuracy: 0.65
9
Mean accuracy: 0.5666666666666667
10
Mean accuracy: 0.36666666666666664
11
Mean accuracy: 0.5666666666666667
12
Mean accuracy: 0.5
13
Mean accuracy: 0.65
14
Mean accuracy: 0.45
15
Mean accuracy: 0.5333333333333333
16
Mean accuracy: 0.5333333333333333
17
Mean accuracy: 0.5666666666666667
18
Mean accuracy: 0.5833333333333333
19
Mean accuracy: 0.55
20
Mean accuracy: 0.5833333333333333
21
Mean accuracy: 0.5166666666666666
22
Mean accuracy: 0.4499999999999999
23
Mean accuracy: 0.5166666666666666
24
Mean accuracy: 0.41666666666666663
25
Mean accuracy: 0.6333333333333333
26
Mean accuracy: 0.4999999999999999
27
Mean accuracy: 0.55
28
Mean accuracy: 0.5
29
Mean accuracy: 0.5666666666666667
30
Mean accuracy: 0.4833333333333334
31
Mean accuracy: 0.65
32

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_AllCpG_Female_100RF.csv", index = False)

0.517


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(30))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.4833333333333333
2
Mean accuracy: 0.41666666666666663
3
Mean accuracy: 0.6666666666666666
4
Mean accuracy: 0.5666666666666667
5
Mean accuracy: 0.5333333333333334
6
Mean accuracy: 0.55
7
Mean accuracy: 0.4666666666666667
8
Mean accuracy: 0.5166666666666666
9
Mean accuracy: 0.6166666666666667
10
Mean accuracy: 0.3666666666666666
11
Mean accuracy: 0.36666666666666664
12
Mean accuracy: 0.5833333333333334
13
Mean accuracy: 0.38333333333333336
14
Mean accuracy: 0.4833333333333334
15
Mean accuracy: 0.5666666666666667
16
Mean accuracy: 0.6166666666666666
17
Mean accuracy: 0.6333333333333333
18
Mean accuracy: 0.4833333333333333
19
Mean accuracy: 0.5
20
Mean accuracy: 0.45
21
Mean accuracy: 0.65
22
Mean accuracy: 0.4666666666666667
23
Mean accuracy: 0.45
24
Mean accuracy: 0.6333333333333334
25
Mean accuracy: 0.6333333333333334
26
Mean accuracy: 0.38333333333333336
27
Mean accuracy: 0.5666666666666667
28
Mean accuracy: 0.5333333333333333
29
Mean accuracy: 0.5666666666666667
30


In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Rand50CpG_Female_100RF.csv", index = False)

0.5193333333333332


## RanFor 100 times. Samples = 98

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(98, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(30))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.4666666666666666
2
Mean accuracy: 0.41666666666666663
3
Mean accuracy: 0.6333333333333333
4
Mean accuracy: 0.5666666666666667
5
Mean accuracy: 0.4833333333333333
6
Mean accuracy: 0.6
7
Mean accuracy: 0.4333333333333333
8
Mean accuracy: 0.4833333333333333
9
Mean accuracy: 0.5833333333333333
10
Mean accuracy: 0.4166666666666667
11
Mean accuracy: 0.4666666666666666
12
Mean accuracy: 0.4833333333333333
13
Mean accuracy: 0.39999999999999997
14
Mean accuracy: 0.5166666666666667
15
Mean accuracy: 0.65
16
Mean accuracy: 0.55
17
Mean accuracy: 0.6333333333333333
18
Mean accuracy: 0.5666666666666667
19
Mean accuracy: 0.4333333333333333
20
Mean accuracy: 0.5166666666666666
21
Mean accuracy: 0.6166666666666667
22
Mean accuracy: 0.4833333333333333
23
Mean accuracy: 0.4333333333333333
24
Mean accuracy: 0.5666666666666667
25
Mean accuracy: 0.5833333333333334
26
Mean accuracy: 0.4
27
Mean accuracy: 0.5166666666666667
28
Mean accuracy: 0.5666666666666667
29
Mean accuracy: 0.583333333

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Rand98CpG_Female_100RF.csv", index = False)

0.521


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Cognition_Boruta_Top50_Data_Female.csv')
print(df.shape)
df.head(2)

(150, 52)


,PATNO,APPRDX,cg11668199,cg04983473,cg25943702,cg12845419,cg01620849,cg22210403,cg21241862,cg19030090,...,cg18497960,cg18830699,cg18319799,cg04954056,cg09902448,cg00319242,cg05125128,cg26819590,cg14587401,cg06191029
0,3002,1,0.045694,0.924158,0.908038,0.518718,0.033501,0.07147,0.904030,0.917645,...,0.031527,0.080178,0.020568,0.924756,0.952915,0.936700,0.894132,0.930234,0.036688,0.622872
1,3003,1,0.050506,0.929821,0.928079,0.548928,0.032749,0.06974,0.914086,0.889673,...,0.028693,0.125536,0.019818,0.906033,0.959074,0.935086,0.887206,0.923072,0.047844,0.895365


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg11668199,cg04983473,cg25943702,cg12845419,cg01620849,cg22210403,cg21241862,cg19030090,cg02232418,...,cg18497960,cg18830699,cg18319799,cg04954056,cg09902448,cg00319242,cg05125128,cg26819590,cg14587401,cg06191029
0,1,0.045694,0.924158,0.908038,0.518718,0.033501,0.07147,0.904030,0.917645,0.830685,...,0.031527,0.080178,0.020568,0.924756,0.952915,0.936700,0.894132,0.930234,0.036688,0.622872
1,1,0.050506,0.929821,0.928079,0.548928,0.032749,0.06974,0.914086,0.889673,0.768925,...,0.028693,0.125536,0.019818,0.906033,0.959074,0.935086,0.887206,0.923072,0.047844,0.895365


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(30))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7833333333333333
2
Mean accuracy: 0.8333333333333334
3
Mean accuracy: 0.7166666666666667
4
Mean accuracy: 0.8166666666666668
5
Mean accuracy: 0.7
6
Mean accuracy: 0.7333333333333334
7
Mean accuracy: 0.7333333333333333
8
Mean accuracy: 0.8
9
Mean accuracy: 0.8666666666666668
10
Mean accuracy: 0.8166666666666668
11
Mean accuracy: 0.85
12
Mean accuracy: 0.7666666666666667
13
Mean accuracy: 0.7666666666666667
14
Mean accuracy: 0.8166666666666667
15
Mean accuracy: 0.8166666666666667
16
Mean accuracy: 0.8166666666666667
17
Mean accuracy: 0.75
18
Mean accuracy: 0.7333333333333333
19
Mean accuracy: 0.7666666666666666
20
Mean accuracy: 0.8166666666666668
21
Mean accuracy: 0.9
22
Mean accuracy: 0.7666666666666667
23
Mean accuracy: 0.8
24
Mean accuracy: 0.7833333333333334
25
Mean accuracy: 0.9
26
Mean accuracy: 0.7833333333333333
27
Mean accuracy: 0.7666666666666666
28
Mean accuracy: 0.7999999999999999
29
Mean accuracy: 0.8166666666666667
30
Mean accuracy: 0.7166666666666667
31

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Boruta50CpG_Female_100RF.csv", index = False)

0.777


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('NMI_Top50_Cognition_Methylome_Female.csv')
print(df.shape)
df.head(2)

(150, 52)


,PATNO,APPRDX,cg20699097,cg12162792,cg25537999,cg03239914,cg14268784,cg10615711,cg20373081,cg06322035,...,cg23865342,cg18304052,cg03166896,cg21409615,cg14074878,cg09351848,cg10644659,cg26713363,cg03795663,cg06512164
0,3002,1,0.522515,0.887998,0.864310,0.024669,0.752473,0.820693,0.485638,0.922618,...,0.896196,0.877310,0.941780,0.882367,0.955485,0.864516,0.913197,0.826201,0.920634,0.854714
1,3003,1,0.504891,0.881857,0.906418,0.027039,0.742175,0.840767,0.514168,0.931757,...,0.876246,0.845442,0.937959,0.893561,0.959934,0.855371,0.896184,0.837343,0.935429,0.825011


In [16]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg20699097,cg12162792,cg25537999,cg03239914,cg14268784,cg10615711,cg20373081,cg06322035,cg17754830,...,cg23865342,cg18304052,cg03166896,cg21409615,cg14074878,cg09351848,cg10644659,cg26713363,cg03795663,cg06512164
0,1,0.522515,0.887998,0.864310,0.024669,0.752473,0.820693,0.485638,0.922618,0.234933,...,0.896196,0.877310,0.941780,0.882367,0.955485,0.864516,0.913197,0.826201,0.920634,0.854714
1,1,0.504891,0.881857,0.906418,0.027039,0.742175,0.840767,0.514168,0.931757,0.203623,...,0.876246,0.845442,0.937959,0.893561,0.959934,0.855371,0.896184,0.837343,0.935429,0.825011


In [17]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(30))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7666666666666667
2
Mean accuracy: 0.8
3
Mean accuracy: 0.8166666666666668
4
Mean accuracy: 0.7
5
Mean accuracy: 0.6333333333333334
6
Mean accuracy: 0.6833333333333333
7
Mean accuracy: 0.7166666666666666
8
Mean accuracy: 0.75
9
Mean accuracy: 0.8
10
Mean accuracy: 0.75
11
Mean accuracy: 0.7166666666666667
12
Mean accuracy: 0.6666666666666667
13
Mean accuracy: 0.7999999999999999
14
Mean accuracy: 0.7333333333333333
15
Mean accuracy: 0.7833333333333334
16
Mean accuracy: 0.7666666666666666
17
Mean accuracy: 0.7666666666666666
18
Mean accuracy: 0.8333333333333334
19
Mean accuracy: 0.75
20
Mean accuracy: 0.7333333333333333
21
Mean accuracy: 0.7666666666666666
22
Mean accuracy: 0.6666666666666667
23
Mean accuracy: 0.6666666666666667
24
Mean accuracy: 0.6666666666666667
25
Mean accuracy: 0.7166666666666667
26
Mean accuracy: 0.7333333333333333
27
Mean accuracy: 0.8333333333333334
28
Mean accuracy: 0.7333333333333334
29
Mean accuracy: 0.7166666666666667
30
Mean accuracy: 0.683

In [18]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_NMI50CpG_Female_100RF.csv", index = False)

0.7408333333333331


## RanFor Top 50 Boruta + NMI 100 times. Samples = 98

In [19]:
df = pd.read_csv('Cognition_Top50sMerge_Data_Female.csv')
print(df.shape)
df.head(2)

(150, 100)


,PATNO,APPRDX,cg20699097,cg12162792,cg25537999,cg03239914,cg14268784,cg10615711,cg20373081,cg06322035,...,cg05015348,cg18497960,cg18830699,cg18319799,cg04954056,cg09902448,cg00319242,cg05125128,cg26819590,cg06191029
0,3002,1,0.522515,0.887998,0.864310,0.024669,0.752473,0.820693,0.485638,0.922618,...,0.925712,0.031527,0.080178,0.020568,0.924756,0.952915,0.936700,0.894132,0.930234,0.622872
1,3003,1,0.504891,0.881857,0.906418,0.027039,0.742175,0.840767,0.514168,0.931757,...,0.929549,0.028693,0.125536,0.019818,0.906033,0.959074,0.935086,0.887206,0.923072,0.895365


In [20]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg20699097,cg12162792,cg25537999,cg03239914,cg14268784,cg10615711,cg20373081,cg06322035,cg17754830,...,cg05015348,cg18497960,cg18830699,cg18319799,cg04954056,cg09902448,cg00319242,cg05125128,cg26819590,cg06191029
0,1,0.522515,0.887998,0.864310,0.024669,0.752473,0.820693,0.485638,0.922618,0.234933,...,0.925712,0.031527,0.080178,0.020568,0.924756,0.952915,0.936700,0.894132,0.930234,0.622872
1,1,0.504891,0.881857,0.906418,0.027039,0.742175,0.840767,0.514168,0.931757,0.203623,...,0.929549,0.028693,0.125536,0.019818,0.906033,0.959074,0.935086,0.887206,0.923072,0.895365


In [21]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(30))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7666666666666667
2
Mean accuracy: 0.85
3
Mean accuracy: 0.8166666666666668
4
Mean accuracy: 0.8
5
Mean accuracy: 0.6833333333333333
6
Mean accuracy: 0.6833333333333333
7
Mean accuracy: 0.7166666666666667
8
Mean accuracy: 0.7999999999999999
9
Mean accuracy: 0.85
10
Mean accuracy: 0.8333333333333333
11
Mean accuracy: 0.7833333333333334
12
Mean accuracy: 0.7666666666666667
13
Mean accuracy: 0.8333333333333333
14
Mean accuracy: 0.9
15
Mean accuracy: 0.7833333333333333
16
Mean accuracy: 0.8166666666666667
17
Mean accuracy: 0.7333333333333334
18
Mean accuracy: 0.8
19
Mean accuracy: 0.7666666666666667
20
Mean accuracy: 0.8333333333333334
21
Mean accuracy: 0.8166666666666667
22
Mean accuracy: 0.8
23
Mean accuracy: 0.8166666666666667
24
Mean accuracy: 0.65
25
Mean accuracy: 0.75
26
Mean accuracy: 0.7166666666666667
27
Mean accuracy: 0.7833333333333333
28
Mean accuracy: 0.7333333333333334
29
Mean accuracy: 0.7666666666666667
30
Mean accuracy: 0.6833333333333333
31
Mean accurac

In [22]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Cognition_Top50sCpG98_Female_100RF.csv", index = False)

0.7749999999999999
